# CCA175 Practice Exam
* Exam    : CCA175
* Title   : CCA Spark and Hadoop Developer Exam
* Vendor  : Cloudera
* Version : V12.35

IT Certification Guaranteed, The Easy Way!

## NO.11 CORRECT TEXT
### Problem Scenario 79 : 
You have been given MySQL DB with following details.

| attribute | value |
|-|-|
| user     |  retail_dba | 
| password |  cloudera   |  
| database |  retail_db | 
| table    |  retail_db.orders | 
| table    |  retail_db.order_items | 
| jdbc URL |  jdbc:mysql://quickstart:3306/retail_db | 

Columns of products table : 

`(product_id | product_categoryid | product_name | product_description | product_price | product_image )`

Please accomplish following activities.

1. Copy "retaildb.products" table to hdfs in a directory p93_products
2. Filter out all the empty prices
3. Sort all the products based on price in both ascending as well as descending order.
4. Sort all the products based on price as well as product_id in descending order.
5. Use the below functions to do data ordering or ranking and fetch top 10 elements 
  * top() 
  * takeOrdered() 
  * sortByKey()

#### Answer

AAT clean simple Solution

Activity 1. Copy "retaildb.products" table to hdfs in a directory p93_products

be careful with commas in string fields. We prefer to specify the fields terminated by *tab *

In [6]:
sqoop import 
--connect jdbc:mysql://quickstart:3306/retail_db 
--username=reatil_dba 
--password=cloudera 
--table=products 
--fields-terminated-by '\t'
--target-dir=p93_products 
-m 1

verify the file was imported

In [8]:
# AAT.
# you can use tail, to see few rows
hdfs dfs -tail p93_products/part-m-00000

#### Activity 2
Filter out all the empty prices

In [10]:
# AAT 
# rdd of tuples
tuples = sc.textFile("p93_products").map( lambda line: line.split( '\t' ) )

# Activity 2. Filter out all the empty prices
rdd = tuples.filter( lambda ( ID, category, name, desc, price, img ): len( price ) > 0 )

#### Activity 3.
Sort all the products based on price in both ascending as well as descending order.

In [12]:
price_kv         = rdd.map( lambda ( ID, category, name, desc, price, img ): ( float(price), name)  )
sorted_price_asc = price_kv.sortByKey( True  )
sorted_price_des = price_kv.sortByKey( False )

#### Activity 4. 
Sort all the products based on price as well as product_id in descending order.

In [14]:
price_ID_kv      = rdd.map( lambda ( ID, category, name, desc, price, img ): ( (float(price), int(ID)), name ) )
sorted_price_ID_des = price_ID_kv.sortByKey( False )

#### Activity 5. 
Use the below functions to do data ordering or ranking and fetch top 10 elements
*    top()
*    takeOrdered()
*    sortByKey()

In [16]:
# the 10 cheapest products 
price_asc_10    = sorted_price_asc.takeOrdered( 10 )

# the 10 more expensive products
price_des_10    = sorted_price_asc.top( 10 )


price_ID_asc = price_ID_kv.sortByKey( True )

# the top 10 products sorted by  price and id in ascending
price_ID_asc_10 = sorted_price_ID_des.takeOrdered( 10 )

# the top 10 products sorted by  price and id in descending
price_ID_des_10 = sorted_price_ID_des.top( 10 )

# the top 10 products sorted by  price asc, and product ID in descending
price_asc_ID_des_10 = price_ID_kv.takeOrdered( 10, key = lambda ( (price, ID), name): (price, -ID) )

#### AAT ... tips

we can specify a parameter as a var or eplicitly as tuple. 
both lines return the same 

`non_empty_lines = rdd.filter( lambda a: len( a[4] ) > 0 )`

`non_empty_lines = rdd.filter( lambda ( ID, category, name, desc, price, img ): len( price ) > 0 )`

if we need to sort a key value paired RDD use `sortByKey()`

if we need to sort an RDD of tuples, we can use `sortBy()` as in the example below
`sorted_price_asc = non_empty_lines.sortBy( lambda a: a[4] )`

`sorted_price_des = non_empty_lines.sortBy( lambda a: a[4], ascending = False )`

end
---